In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('MLT20-21data.txt')
winter2 = pd.read_csv ('MLT21-22data.txt')
winter3 = pd.read_csv ('MLT22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [7]:
# eliminate special weather
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [8]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [10]:
mlt = LocationInfo(
    'Millinocket', 'Maine', 
    latitude= 45.6574,
    longitude= -68.7092,
    timezone='America/Connecticut'
)

mlt

LocationInfo(name='Millinocket', region='Maine', timezone='America/Connecticut', latitude=45.6574, longitude=-68.7092)

In [11]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(mlt.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [12]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(mlt.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [13]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(mlt.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [14]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:55:56  2020-12-01 15:52:00  2020-12-01
1   2020-12-02 06:57:05  2020-12-02 15:51:39  2020-12-02
2   2020-12-03 06:58:12  2020-12-03 15:51:20  2020-12-03
3   2020-12-04 06:59:17  2020-12-04 15:51:05  2020-12-04
4   2020-12-05 07:00:21  2020-12-05 15:50:52  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:21:53  2021-02-24 17:14:45  2021-02-24
86  2021-02-25 06:20:09  2021-02-25 17:16:10  2021-02-25
87  2021-02-26 06:18:24  2021-02-26 17:17:34  2021-02-26
88  2021-02-27 06:16:39  2021-02-27 17:18:58  2021-02-27
89  2021-02-28 06:14:53  2021-02-28 17:20:22  2021-02-28

[90 rows x 3 columns]


In [15]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:55:39  2021-12-01 15:52:05  2021-12-01
1   2021-12-02 06:56:48  2021-12-02 15:51:43  2021-12-02
2   2021-12-03 06:57:56  2021-12-03 15:51:24  2021-12-03
3   2021-12-04 06:59:02  2021-12-04 15:51:08  2021-12-04
4   2021-12-05 07:00:06  2021-12-05 15:50:54  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:22:18  2022-02-24 17:14:24  2022-02-24
86  2022-02-25 06:20:34  2022-02-25 17:15:49  2022-02-25
87  2022-02-26 06:18:50  2022-02-26 17:17:14  2022-02-26
88  2022-02-27 06:17:05  2022-02-27 17:18:38  2022-02-27
89  2022-02-28 06:15:19  2022-02-28 17:20:02  2022-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:55:22  2022-12-01 15:52:10  2022-12-01
1   2022-12-02 06:56:32  2022-12-02 15:51:48  2022-12-02
2   2022-12-03 06:57:39  2022-12-03 15:51:28  2022-12-03
3   2022-12-04 06:58:46  2022-12-04 15:51:11  2022-12-04
4   2022-12-05 06:59:51  2022-12-05 15:50:57  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:22:43  2023-02-24 17:14:04  2023-02-24
86  2023-02-25 06:20:59  2023-02-25 17:15:29  2023-02-25
87  2023-02-26 06:19:15  2023-02-26 17:16:53  2023-02-26
88  2023-02-27 06:17:30  2023-02-27 17:18:18  2023-02-27
89  2023-02-28 06:15:44  2023-02-28 17:19:42  2023-02-28

[90 rows x 3 columns]


In [17]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [18]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MLT,2020-12-02 23:53:00,0.0001,-SN,2020-12-02,2020-12-02 06:57:05,2020-12-02 15:51:39
1,MLT,2020-12-03 00:53:00,0.0001,-SN,2020-12-03,2020-12-03 06:58:12,2020-12-03 15:51:20
2,MLT,2020-12-05 17:53:00,0.7600,-SN BR,2020-12-05,2020-12-05 07:00:21,2020-12-05 15:50:52
3,MLT,2020-12-05 18:53:00,1.0200,-SN BR,2020-12-05,2020-12-05 07:00:21,2020-12-05 15:50:52
4,MLT,2020-12-05 19:53:00,2.2900,SN FG,2020-12-05,2020-12-05 07:00:21,2020-12-05 15:50:52
...,...,...,...,...,...,...,...
185,MLT,2021-02-27 19:53:00,1.0200,+SN FG,2021-02-27,2021-02-27 06:16:39,2021-02-27 17:18:58
186,MLT,2021-02-27 20:53:00,2.0300,-SN BR,2021-02-27,2021-02-27 06:16:39,2021-02-27 17:18:58
187,MLT,2021-02-27 21:53:00,1.0200,-SN BR,2021-02-27,2021-02-27 06:16:39,2021-02-27 17:18:58
188,MLT,2021-02-27 22:53:00,2.7900,-SN BR,2021-02-27,2021-02-27 06:16:39,2021-02-27 17:18:58


In [19]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MLT,2021-12-01 08:53:00,0.0001,-SN BR,2021-12-01,2021-12-01 06:55:39,2021-12-01 15:52:05
1,MLT,2021-12-01 15:53:00,0.0001,-SN,2021-12-01,2021-12-01 06:55:39,2021-12-01 15:52:05
2,MLT,2021-12-01 18:53:00,0.0001,-SN,2021-12-01,2021-12-01 06:55:39,2021-12-01 15:52:05
3,MLT,2021-12-02 12:53:00,0.0001,-SN BR,2021-12-02,2021-12-02 06:56:48,2021-12-02 15:51:43
4,MLT,2021-12-02 13:53:00,0.0001,-SN BR,2021-12-02,2021-12-02 06:56:48,2021-12-02 15:51:43
...,...,...,...,...,...,...,...
211,MLT,2022-02-26 00:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:18:50,2022-02-26 17:17:14
212,MLT,2022-02-27 18:53:00,0.5100,-SN,2022-02-27,2022-02-27 06:17:05,2022-02-27 17:18:38
213,MLT,2022-02-27 19:53:00,0.0001,-SN,2022-02-27,2022-02-27 06:17:05,2022-02-27 17:18:38
214,MLT,2022-02-27 22:53:00,0.0001,-SN BR,2022-02-27,2022-02-27 06:17:05,2022-02-27 17:18:38


In [20]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MLT,2022-12-01 19:53:00,0.0001,-SN,2022-12-01,2022-12-01 06:55:22,2022-12-01 15:52:10
1,MLT,2022-12-13 22:53:00,0.0001,-SN BR,2022-12-13,2022-12-13 07:07:25,2022-12-13 15:50:39
2,MLT,2022-12-14 05:53:00,0.0001,-SN,2022-12-14,2022-12-14 07:08:13,2022-12-14 15:50:49
3,MLT,2022-12-14 06:53:00,0.0001,-SN,2022-12-14,2022-12-14 07:08:13,2022-12-14 15:50:49
4,MLT,2022-12-14 07:53:00,0.0001,-SN BR,2022-12-14,2022-12-14 07:08:13,2022-12-14 15:50:49
...,...,...,...,...,...,...,...
206,MLT,2023-02-26 19:53:00,0.0001,-SN,2023-02-26,2023-02-26 06:19:15,2023-02-26 17:16:53
207,MLT,2023-02-26 20:53:00,0.0001,-SN,2023-02-26,2023-02-26 06:19:15,2023-02-26 17:16:53
208,MLT,2023-02-26 22:53:00,0.0001,-SN BR,2023-02-26,2023-02-26 06:19:15,2023-02-26 17:16:53
209,MLT,2023-02-27 06:53:00,0.0001,-SN,2023-02-27,2023-02-27 06:17:30,2023-02-27 17:18:18


In [21]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
15,MLT,2020-12-06 07:53:00,0.7600,-SN BR,2020-12-06,2020-12-06 07:01:24,2020-12-06 15:50:41
16,MLT,2020-12-06 08:53:00,0.2500,-SN BR,2020-12-06,2020-12-06 07:01:24,2020-12-06 15:50:41
17,MLT,2020-12-06 09:53:00,0.0001,-SN BR,2020-12-06,2020-12-06 07:01:24,2020-12-06 15:50:41
18,MLT,2020-12-06 10:53:00,0.0001,-SN BR,2020-12-06,2020-12-06 07:01:24,2020-12-06 15:50:41
19,MLT,2020-12-06 11:53:00,0.0001,-SN BR,2020-12-06,2020-12-06 07:01:24,2020-12-06 15:50:41
...,...,...,...,...,...,...,...
171,MLT,2021-02-23 08:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 06:23:36,2021-02-23 17:13:20
173,MLT,2021-02-24 06:53:00,0.0001,-SN BR,2021-02-24,2021-02-24 06:21:53,2021-02-24 17:14:45
174,MLT,2021-02-24 07:53:00,0.0001,-SN BR,2021-02-24,2021-02-24 06:21:53,2021-02-24 17:14:45
181,MLT,2021-02-25 09:53:00,0.0001,-SN BR,2021-02-25,2021-02-25 06:20:09,2021-02-25 17:16:10


In [22]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [23]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
16,MLT,2020-12-06 08:53:00,0.2500,-SN BR,2020-12-06,2020-12-06 07:01:24,2020-12-06 15:50:41,2020-12-06 08:01:24,2020-12-06 14:50:41
17,MLT,2020-12-06 09:53:00,0.0001,-SN BR,2020-12-06,2020-12-06 07:01:24,2020-12-06 15:50:41,2020-12-06 08:01:24,2020-12-06 14:50:41
18,MLT,2020-12-06 10:53:00,0.0001,-SN BR,2020-12-06,2020-12-06 07:01:24,2020-12-06 15:50:41,2020-12-06 08:01:24,2020-12-06 14:50:41
19,MLT,2020-12-06 11:53:00,0.0001,-SN BR,2020-12-06,2020-12-06 07:01:24,2020-12-06 15:50:41,2020-12-06 08:01:24,2020-12-06 14:50:41
20,MLT,2020-12-06 13:53:00,0.2500,-SN BR,2020-12-06,2020-12-06 07:01:24,2020-12-06 15:50:41,2020-12-06 08:01:24,2020-12-06 14:50:41
33,MLT,2020-12-10 09:53:00,0.0001,-SN BR,2020-12-10,2020-12-10 07:05:15,2020-12-10 15:50:26,2020-12-10 08:05:15,2020-12-10 14:50:26
34,MLT,2020-12-10 12:53:00,0.0001,-SN BR,2020-12-10,2020-12-10 07:05:15,2020-12-10 15:50:26,2020-12-10 08:05:15,2020-12-10 14:50:26
35,MLT,2020-12-10 13:53:00,0.2500,-SN BR,2020-12-10,2020-12-10 07:05:15,2020-12-10 15:50:26,2020-12-10 08:05:15,2020-12-10 14:50:26
41,MLT,2020-12-18 12:53:00,0.0001,-SN,2020-12-18,2020-12-18 07:11:21,2020-12-18 15:52:07,2020-12-18 08:11:21,2020-12-18 14:52:07
60,MLT,2020-12-31 09:53:00,0.0001,-SN,2020-12-31,2020-12-31 07:15:48,2020-12-31 16:00:36,2020-12-31 08:15:48,2020-12-31 15:00:36


In [24]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [25]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [26]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.4496754716981131
2021-2022: 0.3365965517241379
2022-2023: 0.3637183333333334


In [27]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MLT,2020-12-02 23:53:00,0.0001,-SN,2020-12-02,2020-12-02 06:57:05,2020-12-02 15:51:39
1,MLT,2020-12-03 00:53:00,0.0001,-SN,2020-12-03,2020-12-03 06:58:12,2020-12-03 15:51:20
2,MLT,2020-12-05 17:53:00,0.7600,-SN BR,2020-12-05,2020-12-05 07:00:21,2020-12-05 15:50:52
3,MLT,2020-12-05 18:53:00,1.0200,-SN BR,2020-12-05,2020-12-05 07:00:21,2020-12-05 15:50:52
4,MLT,2020-12-05 19:53:00,2.2900,SN FG,2020-12-05,2020-12-05 07:00:21,2020-12-05 15:50:52
...,...,...,...,...,...,...,...
185,MLT,2021-02-27 19:53:00,1.0200,+SN FG,2021-02-27,2021-02-27 06:16:39,2021-02-27 17:18:58
186,MLT,2021-02-27 20:53:00,2.0300,-SN BR,2021-02-27,2021-02-27 06:16:39,2021-02-27 17:18:58
187,MLT,2021-02-27 21:53:00,1.0200,-SN BR,2021-02-27,2021-02-27 06:16:39,2021-02-27 17:18:58
188,MLT,2021-02-27 22:53:00,2.7900,-SN BR,2021-02-27,2021-02-27 06:16:39,2021-02-27 17:18:58


In [28]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_37540\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [29]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [30]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.6971873015873017
2021-2022: 0.44365333333333346
2022-2023: 0.3269788461538461


In [63]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'ME'

data.to_csv('SO.mlt.csv', index=False)